In [1]:
# Imports
import operator
from collections import Counter
from bitarray import bitarray
import mmh3
import math

# Sampling Task

### Download	the	Honeypot	data (34	GB,	but	we	only	use	the	first	IP	address	values).	Load	one	of	the	files	in	your	favorite	editor	and	describe	in	a	few	lines	what	you	see.

What we see when we open the file in Sublime Text 2 is the following. On the first glimpse it looks quite chaotic.


![title](data/dataset.png)

However, if we looked closer we can indeed see that it is a honeypot shell file. We can observe that the data is timestamped, it includes shell scripts and the protocol with which the files were transfered between the different IPs. The study of files such as this one could potentialy help on identifying certain types of malicious behavior, what the goals of the attackers are or what are the methods they use to attack. Particularly, the information we have is: time, source address, src port, dst address, dst port, and a list of issued commands.


The part of the dataset that we are interested in is the IPs after the date. Thus, we extracted those IPs and we created another file "src_ip.txt" which only contains those IPs in temporal order. 

Let's now count	the number of distinct source IP_addresses.

In [2]:
# Reading the file which contains the source IP addresses (given)-
src_ip = open("data/src_ip.txt", 'r')

# Transforming the file to a list
stream_ips = [line[:-2] for line in src_ip.readlines()]

# Printing the number of distinct source IP addresses
print("The number of distinct IP addresses is: %s" %len(set(stream_ips)))

The number of distinct IP addresses is: 523443


### What are the 10 most frequent IP addresses?

To answer this question we used the module Counter from the package collections. Below, we calculate the frequency of each IP address, then we sort them in descending order and we print them.

In [3]:
# count the frequency of each IP address
freq = Counter(stream_ips)

# Sort them according to their value to find the 13 most frequent ones
sorted_el = sorted(freq.items(), key=operator.itemgetter(1), reverse = True)

# Print the 10 most frequent IP addresses
for i in range(10):
    print (sorted_el[i])

('115.177.11.21', 294690)
('115.176.182.19', 259241)
('192.3.106.4', 66572)
('191.96.249.18', 51251)
('104.192.0.2', 26348)
('198.204.234.2', 22229)
('185.93.185.1', 5157)
('198.204.234.3', 5049)
('115.29.251.11', 3812)
('104.192.0.1', 3701)


### Write code	for	the	FREQUENT	algorithm,	use	it	to	count	the	amounts	in	one	pass

The implementation of the FREQUENT (Misra - Gries) algorithm is presented below.

### What	are	the	10	most	frequent	IP-addresses	and	their	frequencies?
To answer this question by using the FREQUENT algorithm, we ran the FREQUENT algorithm with a value of k equal to a very large number. The reason that we have to pick a large number for k is that since we want to count the frequency of the elements, and not the elements whose frequency exceeds the 1/k fraction of the total count.

We sorted the results of the FREQUENT algorithm. The 10 most frequent IPs of our dataset are presented below in descending order.

Notice that the second number in the resulted tuples is not the actual frequency of each IP (which was found before) but the counted value from the FREQUENT algorithm.

We did that in order to verify that our algorithm works correctly.

In [4]:
# implementation of FREQUENT (Misra - Gries) algorithm
def frequent(stream, k):
    freqD = {}
    for value in stream:
        if value in freqD:
            freqD[value]+=1

        elif len(freqD) < k:
            freqD[value] = 1

        else:
            for existing_value in freqD:
                freqD[existing_value]-=1
            freqD = {x: y for x, y in freqD.items() if y != 0}
        #print(freqD) #(show step-by-step)
    return freqD

In [5]:
# Running the FREQUENT algorithm

# initialize fraction
k = 1000000

# Find elements whose frequency exceeds 1/k fraction of the total count
elements = frequent(stream_ips, k)

# Sort them according to their value to find the 13 most frequent ones
sorted_freq = sorted(elements.items(), key=operator.itemgetter(1), reverse = True)

print("The 10 most frequent IP addresses are:")

for i in range(10):
    print(sorted_freq[i])
    

The 10 most frequent IP addresses are:
('115.177.11.21', 294690)
('115.176.182.19', 259241)
('192.3.106.4', 66572)
('191.96.249.18', 51251)
('104.192.0.2', 26348)
('198.204.234.2', 22229)
('185.93.185.1', 5157)
('198.204.234.3', 5049)
('115.29.251.11', 3812)
('104.192.0.1', 3701)


As could be observed from the above results by using a very large value for k we get the exact same results as when we calculated the frequencies in the normal way.

This is reasonable, as the condition of the FREQUENT algorithm "elif len(freqD) < k-1:" is always true. Thus, our algorithm basically just keeps a dictionary with the IPs  as keys, and increases their values as it processes the IP stream.

### Run	FREQUENT	using	reservoirs	of	size	10,	100,	and	1000.	What	are	the	10	most	frequent	IP-addresses	and	their	frequencies?

The code below runs the FREQUENT algorithm with k equal to 10, 100 and 1000.

In [15]:
# Running the FREQUENT algorithm

# initialize fraction
for k in [10, 100, 1000]:
    
    print("\n")
    # Find elements whose frequency exceeds 1/k fraction of the total count
    elements = frequent(stream_ips, k)

    # Sort them according to their value to find the 13 most frequent ones
    sorted_freq = sorted(elements.items(), key=operator.itemgetter(1), reverse = True)

        
    if len(sorted_freq)<10:
        num = len(sorted_freq)
    else:
        num = 10
        
    print("The %s most frequent IP addresses with k = %s are: \n (On the right the real 10 most frequent IP addresses are presented)\n" %(num,k))

    for i in range(num):
        print("FREQUENT #%s: %s\t \t REAL #%s: %s" %(i, sorted_freq[i], i, sorted_el[i]))
        
        # sorted_el contains the real frequencies of the IPs.
        # print it  as well, if you want to see the differences in the same page.
        #print(sorted_el[i])
  




The 8 most frequent IP addresses with k = 10 are: 
 (On the right the real 10 most frequent IP addresses are presented)

FREQUENT #0: ('115.176.182.19', 79417)	 	 REAL #0: ('115.177.11.21', 294690)
FREQUENT #1: ('191.96.249.18', 696)	 	 REAL #1: ('115.176.182.19', 259241)
FREQUENT #2: ('187.199.79.1', 2)	 	 REAL #2: ('192.3.106.4', 66572)
FREQUENT #3: ('114.32.130.1', 1)	 	 REAL #3: ('191.96.249.18', 51251)
FREQUENT #4: ('115.108.40.1', 1)	 	 REAL #4: ('104.192.0.2', 26348)
FREQUENT #5: ('87.96.167.23', 1)	 	 REAL #5: ('198.204.234.2', 22229)
FREQUENT #6: ('187.23.203.25', 1)	 	 REAL #6: ('185.93.185.1', 5157)
FREQUENT #7: ('82.178.61.9', 1)	 	 REAL #7: ('198.204.234.3', 5049)


The 10 most frequent IP addresses with k = 100 are: 
 (On the right the real 10 most frequent IP addresses are presented)

FREQUENT #0: ('115.177.11.21', 266748)	 	 REAL #0: ('115.177.11.21', 294690)
FREQUENT #1: ('115.176.182.19', 242626)	 	 REAL #1: ('115.176.182.19', 259241)
FREQUENT #2: ('192.3.106.4', 43

As expected, the results are not the same as before. 

#### k = 10

With k = 10 the algorithm does not even return 10 values as it is looking for the IPs whose frequency exceeds the fraction of 10% over the total count of IPs. Apparently, only 7 IPs fulfill this requirement. 

In addition, only two of those IPs actually exist in the true 7 most frequent IPs. Those results are reasonable, as there are various IPs which have a frequency 1 according to the FREQUENT algorithm, and the sorting of them by value does not end up with the correct order. Furthermore, depending on the initial stream it is possible for a high frequently value to be decreased a lot of times (e.g if it has been increased a lot initially, and then the dictionary is full and then different IPs constantly appear) and eventually obtain a smaller frequency from others even if in reality this is not correct.

#### k = 100

The results in this case are closer to the ground truth. This is logical as the size of the reservoir is larger and the decreasing happens less often than before.

#### k =  1000

Finally, in that case the order is the same as the initial. Still, the frequencies are not the same (as they were for example with the very large k) but the ranking is.

In general, the larger the size of the reservoirs the less times the decreasing occurs. Without the decreasing part, the algorithm's behaviour is the same as the simple counter.

# Hashing Task
Splitting	the	honeypot	data	50:50	into	split	(first	half) and	test	(second	half)	maintaining	the	temporal	order.	

In [18]:
# stream_ips is a list containing the IPs in temporal order

train = stream_ips[:int(len(stream_ips)/2)]
test = stream_ips[int(len(stream_ips)/2):]

# Check size of train and test set
print("The size of the initial set is: %s" %len(stream_ips))
print("The size of the training set is: %s" %len(train))
print("The size of the testing set is: %s" %len(test))

The size of the initial set is: 3414011
The size of the training set is: 1707005
The size of the testing set is: 1707006


### Test	for	each	source	IP in	the	test	set	whether	it	occurs	in	the	train	set.

We store the real number of source IPs in the test set which also occur in the train set in the variable named gt. We will use this variable later.


In [20]:
# We firstly create two sets from our training and testing lists
train_set = set(train)
test_set = set(test)

# Then we check the intersection of the two sets
inter = set.intersection(train_set, test_set)

# The real number of source IPs in the test set which also occur in the train set
gt = len(inter) # our ground truth
print("There are %s source IPs in the test set which also occur in the train set"%gt)

There are 58061 source IPs in the test set which also occur in the train set


### Write	code	for	a	BLOOM	filter

In [21]:
class BloomFilter:
    
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)
        
    def add(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.size
            self.bit_array[result] = 1
        
    def lookup(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.size
            if self.bit_array[result] == 0:
                return "Nope"
        return "Probably"



Now, let's see what we can do with the bloom filter.


By using the following formulas we are able to compute the size of the bloom filter and number of the hash functions to be used, assuming that we now the expected number of items and the desired false positive probability.

![title](data/k.png)

![title](data/m.png) 

We realized, that if we use the expected number of items equal to the length of the train set and a probability of 0.000001 we get the correct results (58061). Run the code below to check the bit array size and the number of hash functions. 

In [22]:
n = len(train_set) # expected number of items
p = 0.000001 # probability percentage
m = - (n* math.log(p)) /(math.log(2)**2) # size of the bit array
k = (m/n)* math.log(2) # number of hash functions

m = round(m)
k = round(k)

print("Bit array size: %s" %m)
print("Number of hash functions: %s" %k)

bf = BloomFilter(m, k)

# Adding training set's IPs
for ip in train_set:
    bf.add(ip)

print("IPs from the training set added")

# lookup for the IPs in the test set
count = 0
for ip in test_set:
    if bf.lookup(ip) == "Probably":
        count+=1
print("There are no more than %s source IPs in the test set which also occur in the train set"%count)

Bit array size: 9776213
Number of hash functions: 20
IPs from the training set added
There are no more than 58061 source IPs in the test set which also occur in the train set


### Perform	the	same	test	using	BLOOM	filters	of	sizes	10,	100,	and	1000.	Explain	the	 differences	using	the	theoretical	false	positive	rates.

For that purpose we will use different sizes of the bit array and then derive the number of k.

Again, n equals to the size of the training set.

Running the code below, we found a problem. The number of hash functions, defined by the formulas, is always 0.

In [23]:
n = len(train_set) # expected number of items

results = {} # dictionary with the results for different bit array sizes

for m in [10, 100, 1000]:
    k = (m/n)* math.log(2) # number of hash functions
    k = round(k)
    print("Running for bit array size: %s" %m)
    print("Number of hash functions: %s \n" %k)
    bf = BloomFilter(m, k)
 
    # Adding training set's IPs
    for ip in train_set:
        bf.add(ip)

    # lookup for the IPs in the test set
    
    for ip in test_set:
        if bf.lookup(ip) == "Probably":
            if m in results:
                results[m].append(ip)
            else:
                results[m] = [ip]

    

Running for bit array size: 10
Number of hash functions: 0 

Running for bit array size: 100
Number of hash functions: 0 

Running for bit array size: 1000
Number of hash functions: 0 



As it could be observed from the above results, for such a small size of the bit array the number of the hash functions which is needed is 0. Thus, to get some results we should increase the size significantly.

After experimentation we found out that to get at least one hash function we should use a size of around 250.000. Thus, we decided to try the values 500.000, 1000.000, 2.000.000, 4.000.000, 8.000.000, 10.000.000.

In [24]:
n = len(train_set) # expected number of items

results = {} # dictionary with the results for different bit array sizes
fpr = {}  # dictionary with the false positive rates for different bit array sizes

# for each bit array size
for m in [500000, 1000000, 2000000, 4000000, 8000000, 10000000]:
    # calculate k
    k = (m/n)* math.log(2) # number of hash functions
    k = round(k)
    print("Running for bit array size: %s" %m)
    print("Number of hash functions: %s" %k)
    
    # define bloom filter
    bf = BloomFilter(m, k)
 
    # Adding training set's IPs
    for ip in train_set:
        bf.add(ip)

    # lookup for the IPs in the test set
    # and if they exist in the training set store them in results
    for ip in test_set:
        if bf.lookup(ip) == "Probably":
            if m in results:
                results[m].append(ip)
            else:
                results[m] = [ip]
    
    # False positive rate
    # The total number of testing set's IPs we said they possibly exist in the training set minus 
    # the actual number of the co-occured IPs. With the bloom filter always we get more or equal number of IPs 
    # than the real number. This is happening due to the fact that if  an IP exists in both sets, bloom filter will
    # definetly find it. The case where bloom filter might be wrong is that it could "say" that an IP exists 
    # in the training set while in reality it does not. It could never "say" that an IP that does not exist in the
    #training set, exists. (Basically, there are no FN, only FP)
    
    # How many IPs of the test set are not in the training set but we said they are?
    # Number of bloom filter results minus the correct number (ground truth)
    fp = len(results[m]) - gt
    tn = len(test_set) - len(results[m])
    fpr[m] = fp/(fp+tn)
    print("The number of test set's IPs which are possibly in the training set are: %s" %len(results[m]))
    print("The false positive rate is: %s \n" %fpr[m])
    
    

Running for bit array size: 500000
Number of hash functions: 1
The number of test set's IPs which are possibly in the training set are: 148178
The false positive rate is: 0.4912025378552507 

Running for bit array size: 1000000
Number of hash functions: 2
The number of test set's IPs which are possibly in the training set are: 102500
The false positive rate is: 0.24222454786277267 

Running for bit array size: 2000000
Number of hash functions: 4
The number of test set's IPs which are possibly in the training set are: 68862
The false positive rate is: 0.05887322715330695 

Running for bit array size: 4000000
Number of hash functions: 8
The number of test set's IPs which are possibly in the training set are: 58671
The false positive rate is: 0.0033249392244715527 

Running for bit array size: 8000000
Number of hash functions: 16
The number of test set's IPs which are possibly in the training set are: 58066
The false positive rate is: 2.72536002005865e-05 

Running for bit array size: 100

### Explain	the	differences	using	the	theoretical	false	positive	rates.

The theoretical false positive rates are given by the following formula:

![title](data/fp_def.png) 

To count them and compare them with our results we wrote the following script which takes into account the above bit array sizes and the corresponding number of hash functions.

In [25]:
for tup in [(500000,1), (1000000,2), (2000000,4), (4000000,8), (8000000,16), (10000000,20)]:
    m = tup[0]
    k = tup[1]
    fp = (1 - ( 1 - (1.0/m))**(k*n))**k
    
    print("Running for bit array size: %s" %m)
    print("Number of hash functions: %s" %k)
    print("The theoretical false positive rate is: %s " %fp)
    print("Difference between the theoretical and real false positive rate is: %s \n" %(abs(fp-fpr[m])))



Running for bit array size: 500000
Number of hash functions: 1
The theoretical false positive rate is: 0.4933641003074374 
Difference between the theoretical and real false positive rate is: 0.0021615624521866827 

Running for bit array size: 1000000
Number of hash functions: 2
The theoretical false positive rate is: 0.24340796553013969 
Difference between the theoretical and real false positive rate is: 0.001183417667367015 

Running for bit array size: 2000000
Number of hash functions: 4
The theoretical false positive rate is: 0.05924739629536252 
Difference between the theoretical and real false positive rate is: 0.0003741691420555693 

Running for bit array size: 4000000
Number of hash functions: 8
The theoretical false positive rate is: 0.003510251521083801 
Difference between the theoretical and real false positive rate is: 0.0001853122966122482 

Running for bit array size: 8000000
Number of hash functions: 16
The theoretical false positive rate is: 1.232185713747658e-05 
Differ

As shown by the above results, the differences between the theoretical and actual false positive rates values are very small. Particularly, less than 0.0022 for all the cases.

The difference between the theoretical and real values was expected. The formula for the theoretical computation of the false positive rate is based on the assumption that the probabilities of each bit being set is independent (https://en.wikipedia.org/wiki/Bloom_filter). However, it is a close approximation of the real values.

###  Extend	the	BLOOM	filter	to	a	Count-Min	sketch,	play	with	different	heights	and	widths	for	the	CM	sketch	matrix.

In the same logic with the bloom filter class we created the count-min class.

In [26]:
class CountMin:
    
    def __init__(self, w, d):
        self.size = w*d
        self.w = w
        self.hash_count = d
        self.cm_array =  [[0]*w for i in range(d)]
        
    def add(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.w
            self.cm_array[seed][result] += 1
        
    def point(self, string):
        min = 1000000000000
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.w
            if self.cm_array[seed][result]<min:
                min = self.cm_array[seed][result]
        return min
    #def lookup(self, string):
    #    for seed in range(self.hash_count):
    #        result = mmh3.hash(string, seed) % self.size
    #        if self.bit_array[result] == 0:
    #            return "Nope"
    #    return "Probably"


### Play	with	different	heights	and	widths	for	the	CM	sketch	matrix

The count-min sketch's parameters w and d can be chosen by setting w = ⌈e/ε⌉ and d = ⌈ln 1/δ⌉, where the error in answering a query is within an additive factor of ε with probability 1 − δ , and e is Euler's number [https://en.wikipedia.org/wiki/Count%E2%80%93min_sketch]. Thus, we decided to run the algorithm with different values of epsilon and delta which lead to different values of height and width. 

With the following script after creating the count-min data structure for the different epsilon and delta, we add the IPs to the structure and then we find the frequency of each IP and we store it in a dictionary (cm_res).

Finally, we calculate and present the summation of the absolute difference between the count-min frequency of the 10 most frequent IPs and their actual (ground truth) frequency.

In [27]:
e = 2.718281828

#initialize ε and δ
for epsilon in [0.0001, 0.001, 0.01]:
    for delta in [0.0001, 0.001, 0.01]:
        # derive w and d
        w = round(e/epsilon)
        d = round(math.log(1/delta))

        # create count min
        cm = CountMin(w, d)

        # streaming and adding to matrix
        for ip in stream_ips:
                cm.add(ip)
                
        # find frequency and store it to cm_res
        cm_res = {}
        for ip in stream_ips:
            cm_res[ip] = cm.point(ip)

        # Sort them according to their value to find the 10 most frequent ones
        sorted_cm = sorted(cm_res.items(), key=operator.itemgetter(1), reverse = True)

        diff = 0
        for i in range(10):
            diff+= abs(sorted_cm[i][1] - sorted_el[i][1])

        print("ε = %s | δ = %s | w = %s | d = %s " %(epsilon, delta, w, d))
        print("The total difference between the frequency of the 10 most frequent results of count min and the ground truth: \n%s \n"%diff)    

ε = 0.0001 | δ = 0.0001 | w = 27183 | d = 9 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
383 

ε = 0.0001 | δ = 0.001 | w = 27183 | d = 7 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
407 

ε = 0.0001 | δ = 0.01 | w = 27183 | d = 5 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
445 

ε = 0.001 | δ = 0.0001 | w = 2718 | d = 9 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
7089 

ε = 0.001 | δ = 0.001 | w = 2718 | d = 7 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
7387 

ε = 0.001 | δ = 0.01 | w = 2718 | d = 5 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
7872 

ε = 0.01 | δ = 0.0001 | w = 272 | d =

### Does	it	provide	better	approximations	than	the	FREQUENT	algorithm?	Use	the	theory	to	explain	why	(not)

In general, by tuning the parameters we can get similar results with both methods. If we stretch them enough we could also get the actual correct results. However, depending the limitations of time and memory we migth use the one or the other. 

The FREQUENT algorithm underestimates the frequency while the count min overestimates it.

In addition, FREQUENT algorithm seems to use less memory but it focus more in the order of the frequency ( find the x most frequent elements) and not so much on the absolute frequency number. 
